### Part d): Classification  analysis using neural networks

With a well-written code it should now be easy to change the
activation function for the output layer.

Here we will change the cost function for our neural network code
developed in parts b) and c) in order to perform a classification analysis. 

We will here study the Wisconsin Breast Cancer  data set. This is a typical binary classification problem with just one single output, either True or Fale, $0$ or $1$ etc.
You find more information about this at the [Scikit-Learn
site](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_breast_cancer.html) or at the [University of California
at Irvine](https://archive.ics.uci.edu/ml/datasets/breast+cancer+wisconsin+(original)). 

To measure the performance of our classification problem we use the
so-called *accuracy* score.  The accuracy is as you would expect just
the number of correctly guessed targets $t_i$ divided by the total
number of targets, that is

$$
\text{Accuracy} = \frac{\sum_{i=1}^n I(t_i = y_i)}{n} ,
$$

where $I$ is the indicator function, $1$ if $t_i = y_i$ and $0$
otherwise if we have a binary classification problem. Here $t_i$
represents the target and $y_i$ the outputs of your FFNN code and $n$ is simply the number of targets $t_i$.

Discuss your results and give a critical analysis of the various parameters, including hyper-parameters like the learning rates and the regularization parameter $\lambda$ (as you did in Ridge Regression), various activation functions, number of hidden layers and nodes and activation functions.  

As stated in the introduction, it can also be useful to study other
datasets. 

Again, we strongly recommend that you compare your own neural Network
code for classification and pertinent results against a similar code using **Scikit-Learn**  or **tensorflow/keras** or **pytorch**.

PLAN:
1. last ned breast cancer data set
2. bruk ffnn til klassifikasjon (med ... som siste lag)
3. Bruk back propogation for å forbedre svaret
4. bruke accuracy til å teste resultatene mine 

In [23]:
## just downloading the dataset
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import autograd.numpy as np
from autograd import grad, elementwise_grad
import FFNN as fn

wisconsin = load_breast_cancer()
X = wisconsin.data
target = wisconsin.target
target = target.reshape(target.shape[0], 1)

X_train, X_val, t_train, t_val = train_test_split(X, target)

scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_val = scaler.transform(X_val)

#print(X[0].size)

#""""
network_input_size = X[0].size
print(X.size)
print(network_input_size)
layer_output_sizes = [2, 3, 2]
activation_funcs = [fn.sigmoid, fn.sigmoid, fn.ReLU]
activation_ders = [fn.sigmoid_der, fn.sigmoid_der, fn.ReLU_der]

layers = [(np.random.randn(network_input_size,2), np.random.randn(network_input_size))]
batched_layers = [(layers[0][0].T, layers[0][1])]
##fn.create_layers(network_input_size, layer_output_sizes)

predict = fn.feed_forward_batch(X, layers, activation_funcs)

layer_grads = fn.backpropagation_batch(X, layers, activation_funcs, target, activation_ders)
print(layer_grads)

cost_grad = grad(fn.cost, 0)
print(cost_grad(layers, X, activation_funcs, target))

cost_grad = grad(fn.cost_batch, 0)
print(cost_grad(batched_layers, X, activation_funcs, target))
#"""

30


ValueError: operands could not be broadcast together with shapes (569,2) (30,) 